# HR IPM Data

This notebook describes how the HR data is downloaded, and transformed into a form suitable for further analysis.

There is also a little light characterisation, in terms of some of the major feature fields.

## Setup

In [ ]:
import pandas
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.palettes import Category20, Category20b

In [ ]:
output_notebook()

In [ ]:
import json

In [ ]:
secrets = json.load(open("./secrets/secrets.json"))

In [ ]:
from db_utils import minio_utils

## Downloading Datafiles
Datafile has been provided as Excel spreadsheet, placed in our Edge Minio. Need to download, and put into the `tempdata` directory.

In [ ]:
minio_utils.minio_to_file(
    "tempdata/Full KPA File.xlsx",
    minio_bucket="writeonly",
    minio_key=secrets["minio"]["edge"]["access"],
    minio_secret=secrets["minio"]["edge"]["secret"],
    data_classification=minio_utils.DataClassification.EDGE
)

In [ ]:
minio_utils.minio_to_file(
    "tempdata/data_influencer_list.txt",
    minio_bucket="writeonly",
    minio_key=secrets["minio"]["edge"]["access"],
    minio_secret=secrets["minio"]["edge"]["secret"],
    data_classification=minio_utils.DataClassification.EDGE
)

In [ ]:
data_strategy_team_members_df = minio_utils.minio_to_dataframe(
    "data-strategy-team-members",
    minio_key=secrets["minio"]["edge"]["access"],
    minio_secret=secrets["minio"]["edge"]["secret"],
    data_classification=minio_utils.DataClassification.EDGE
)

In [ ]:
data_strategy_dcc_members_df = minio_utils.minio_to_dataframe(
    "data-strategy-dcc-members",
    minio_key=secrets["minio"]["edge"]["access"],
    minio_secret=secrets["minio"]["edge"]["secret"],
    data_classification=minio_utils.DataClassification.EDGE
)

In [ ]:
data_strategy_dcc_members_df.sample(10)

## Constructing list of Data Strategy Associated Names

In [ ]:
with open('tempdata/data_influencer_list.txt') as data_influencer_file:
    data_influencer_list = data_influencer_file.read().split("\n")
    
data_influencer_lower_list = pandas.Series(
    map(lambda x: x.lower(), data_influencer_list)
)

In [ ]:
data_strategy_affliated = pandas.DataFrame(
    data_influencer_lower_list.append([
        data_strategy_team_members_df.Name.str.lower(),
        data_strategy_dcc_members_df.Name.str.lower()
    ]).unique(),
    columns=("Name",)
)

In [ ]:
minio_utils.dataframe_to_minio(
    data_strategy_affliated,
    minio_bucket="data-strategy-affliated",
    minio_key=secrets["minio"]["confidential"]["access"],
    minio_secret=secrets["minio"]["confidential"]["secret"],
    data_classification=minio_utils.DataClassification.CONFIDENTIAL,
    file_format="pickle.gz"
)

## Converting HR Data to Dataframe and uploading

In [ ]:
kpa_df = pandas.read_excel('./tempdata/Full KPA File.xlsx')

### Data Munging

In [ ]:
print(kpa_df.iloc[1].values)

Removing some unnecessary rows at beginning of spreadsheet:

In [ ]:
formatted_kpa_df = kpa_df.drop(
    kpa_df.iloc[:2].index
)

Setting column names, without spaces:

In [ ]:
formatted_kpa_df.columns = (
    'Directorate',
    'Department',
    'EmployeeNumber',
    'EmployeeName',
    'PositionNumber',
    'PositionName',
    'PayScaleGroup',
    'Template',
    'CriteriaGroup',
    'Criterion',
    'Row',
    'AppraisalScoreWeight'
)

In the `Row` field, often spaces are inserted between L and the number, e.g. `L 1`. Removing those spaces:

In [ ]:
formatted_kpa_df.Row = formatted_kpa_df.Row.str.replace(r'L\s*(?P<level>\d+)', 
                                                        lambda m: 'L' + m.group('level'))

Extracting the T Level into a separate, standalone field:

In [ ]:
formatted_kpa_df['TLevel'] = formatted_kpa_df.PayScaleGroup.str.extract(r'T(\d+).*').astype(float)

### Uploading

In [ ]:
minio_utils.dataframe_to_minio(
    formatted_kpa_df,
    minio_bucket="hr-ipm-data",
    minio_key=secrets["minio"]["confidential"]["access"],
    minio_secret=secrets["minio"]["confidential"]["secret"],
    data_classification=minio_utils.DataClassification.CONFIDENTIAL,
    file_format="pickle.gz"
)

## HR Data Characterisation

Making sure we're on the same page:

In [ ]:
formatted_kpa_df = minio_utils.minio_to_dataframe(
    minio_bucket="hr-ipm-data",
    minio_key=secrets["minio"]["confidential"]["access"],
    minio_secret=secrets["minio"]["confidential"]["secret"],
    data_classification=minio_utils.DataClassification.CONFIDENTIAL
)

### Tabular Form

In [ ]:
formatted_kpa_df.groupby(['Directorate', 'Department', 'PositionName']).PositionNumber.nunique().to_frame().reset_index().sort_values(
    by=['Directorate', 'Department', 'PositionNumber'],
    ascending=False
).to_html('./report/hr_data_summary_table.html', index=False)

### Plotting

In [ ]:
def get_counts(data_df):
    directorate_counts = data_df.groupby(['Directorate']).EmployeeNumber.nunique().sort_values(ascending=False)
    department_counts = data_df.groupby(['Department']).EmployeeNumber.nunique().sort_values(ascending=False)
    tlevel_counts = data_df.groupby(['TLevel']).EmployeeNumber.nunique()#.sort_values(ascending=False)
    position_counts = data_df.groupby(['PositionName']).EmployeeNumber.nunique().sort_values(ascending=False)
    
    return directorate_counts, department_counts, tlevel_counts, position_counts

In [ ]:
def generate_count_plot(count_series, x_axis_label, title, category_limit = None, ymax=None,
                        width=400, height=600, tools=["save"], colour_pallete=Category20):
    title = f"{title} (top {category_limit})" if category_limit is not None else title
    
    category_limit = len(count_series) if category_limit is None else category_limit
    ymax = count_series.max()*1.05 if ymax is None else ymax*1.05
    
    count_figure = figure(width=width, plot_height=height, title=title, 
                          x_range=count_series.index.values[:category_limit].astype(str), y_range=(0, ymax),
                          tools=TOOLS)
        
    count_figure.vbar(
        count_series.index.values[:category_limit].astype(str), top=count_series.values[:category_limit], 
        width=0.9, color=colour_pallete[category_limit]
    )
    count_figure.xaxis.major_label_orientation = "vertical"
    count_figure.xaxis.axis_label = x_axis_label
    
    return count_figure

In [ ]:
def generate_overview_plot(data_df, output_plot_path, width=400, height=600):
    directorate_counts, department_counts, tlevel_counts, position_counts = get_counts(data_df)
    
    output_file(output_plot_path, mode="cdn")
    
    department_figure = generate_count_plot(
        directorate_counts, "Directorate", "Directorate Breakdown"
    )
    directorate_figure = generate_count_plot(
        directorate_counts, "Department", "Department Breakdown", 15
    )
    tlevel_figure = generate_count_plot(
        tlevel_counts, "T-Level", "T-Level Breakdown",
    )
    positions_figure = generate_count_plot(
        position_counts, "Positions", "Position Breakdown", 15
    )

    # show the results
    show(gridplot([
        [tlevel_figure, positions_figure],
        [directorate_figure, department_figure],
    ]))

In [ ]:
generate_overview_plot(formatted_kpa_df, "report/hr_data_summary.html")

## Splitting out Administrative Positions

At the request of the Data Capabilities workstream, I'm splitting out the administrative positions, which make up about 30% of the dataset.

In [ ]:
_, _, _, position_counts = get_counts(formatted_kpa_df)

In [ ]:
position_counts.iloc[:20]

In [ ]:
admin_positions = {
    "clerk", "senior clerk", "specialist clerk",
    "assistant administrative officer", "administrative officer 1", "administrative officer 2", "administrative officer 3",
    "secretary", "executive personal assistant", "senior secretary"
}

In [ ]:
position_counts[
    position_counts.index.str.lower().isin(admin_positions)
].sum()/position_counts.sum()

In [ ]:
generate_overview_plot(
    formatted_kpa_df.query("PositionName.str.lower().isin(@admin_positions)"), 
    "report/hr_admin_data_summary.html"
)

In [ ]:
generate_overview_plot(
    formatted_kpa_df.query(
        "~(PositionName.str.lower().isin(@admin_positions))"
    ), 
    "report/hr_non_admin_data_summary.html"
)

In [ ]:
def generate_comparison_plot(data_df, output_plot_path, admin_positions, width=400, height=600):
    admin_df = data_df.query(
        "(PositionName.str.lower().isin(@admin_positions))"
    )
    nonadmin_df = data_df.query(
        "~(PositionName.str.lower().isin(@admin_positions))"
    )
    
    admin_directorate_counts, admin_department_counts, admin_tlevel_counts, admin_position_counts = get_counts(admin_df)
    na_admin_directorate_counts, na_admin_department_counts, na_admin_tlevel_counts, na_admin_position_counts = get_counts(nonadmin_df)
    
    output_file(output_plot_path, mode="cdn")
    
    figures = [
        [
            generate_count_plot(admin_count, count_type, f"{count_type} Admin Breakdown", cat_limit, ymax=max(admin_count.max(), na_admin_count.max()), 
                                width=width, height=height, colour_pallete=Category20b),
            generate_count_plot(na_admin_count, count_type, f"{count_type} Non-Admin Breakdown", cat_limit, ymax=max(admin_count.max(), na_admin_count.max()), 
                                width=width, height=height, colour_pallete=Category20b)
        ]
        for admin_count, na_admin_count, count_type, cat_limit in
        (
            (admin_directorate_counts, na_admin_directorate_counts, "Directorate", None),
            (admin_department_counts, na_admin_department_counts, "Department", 15),
            (admin_tlevel_counts, na_admin_tlevel_counts, "T-Level", None),
            (admin_position_counts, na_admin_position_counts, "Position", 15)
        )
    ]

    # show the results
    show(gridplot(
        figures
    ))

In [ ]:
generate_comparison_plot(formatted_kpa_df, "report/hr_admin_data_comparison.html", admin_positions)

In [ ]:
minio_utils.dataframe_to_minio(
    formatted_kpa_df.query(
        "PositionName.str.lower().isin(@admin_positions))"
    ),
    minio_bucket="hr-ipm-data-admin",
    minio_key=secrets["minio"]["confidential"]["access"],
    minio_secret=secrets["minio"]["confidential"]["secret"],
    data_classification=minio_utils.DataClassification.CONFIDENTIAL,
    file_format="pickle.gz"
)

In [ ]:
minio_utils.dataframe_to_minio(
    formatted_kpa_df.query(
        "~(PositionName.str.lower().isin(@admin_positions))"
    ),
    minio_bucket="hr-ipm-data-non-admin",
    minio_key=secrets["minio"]["confidential"]["access"],
    minio_secret=secrets["minio"]["confidential"]["secret"],
    data_classification=minio_utils.DataClassification.CONFIDENTIAL,
    file_format="pickle.gz"
)